In [1]:
import cv2  
import base64
from openai import OpenAI

client = OpenAI(api_key='sk-REDACTED')

def query_model(video_path, prompt):
    video = cv2.VideoCapture(video_path)
    
    base64Frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

    PROMPT_MESSAGES = [
        {
            "role": "user",
            "content": [
                prompt,
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::100000]),
            ],
            'detail': 'high',
        },
    ]
    params = {
        "model": "gpt-4o-mini",
        "messages": PROMPT_MESSAGES,
    }

    result = client.chat.completions.create(**params)
    return result.choices[0].message.content

In [2]:
prompt = '''
summarize video
'''

video_path = '/Users/chipwy/Dev/saas-delta/resources/videos/Fire Extinguisher Safety - AFIRE100.mp4'
response = query_model(video_path, prompt)
response


'The video showcases a learning management system interface, specifically focusing on a training module related to fire safety. It demonstrates how to search for and access a course titled "Fire Extinguisher Safety" (code: AFIRE100) within the Delta library. The course is categorized as free, and users are prompted to start the course from the available options. The overall aim is to provide safety training specifically about using fire extinguishers.'